In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
con = sqlite3.connect('db')

In [3]:
!gdown --id 1Vzzp3iopwUHnznj0Zf0DVu6uUwwqxtEL

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Vzzp3iopwUHnznj0Zf0DVu6uUwwqxtEL
To: /content/german_credit_augmented.csv
100% 69.6k/69.6k [00:00<00:00, 87.5MB/s]


In [4]:
df = pd.read_csv('/content/german_credit_augmented.csv')

In [5]:
df['contract_dt'] = pd.to_datetime(df['contract_dt'],format='%Y-%m-%d %H:%M:%S')

In [6]:
df.to_sql('german_credit',con,index=False,if_exists='replace')

1000

In [7]:
def select(sql):
  return pd.read_sql(sql,con)

In [8]:
sql = '''
select
t.sex,

count(*) as cnt,
avg(t.credit_amount) as credit_amount_avg

 from german_credit t
 group by t.sex
 '''

In [9]:
select(sql)

,sex,cnt,credit_amount_avg
0,female,310,2877.774194
1,male,690,3448.040580


In [10]:
df.groupby('sex')['credit_amount'].agg(['count','mean'])

,count,mean
sex,,
female,310,2877.774194
male,690,3448.040580


In [11]:
sql = '''select count(distinct t.housing), count(t.housing) from german_credit t'''
select(sql)

,count(distinct t.housing),count(t.housing)
0,3,1000


In [14]:
sql = '''select
sum(case when t.checking_account is null then 1 else 0 end) as is_null
from german_credit t'''

select(sql)

,is_null
0,394


avg, count(column), sum - пропуски не учитывает, count(*) - учитывает, решение - coalesce()

In [16]:
sql = '''select

t.checking_account,

coalesce(t.checking_account,'no_info')

from german_credit t'''

select(sql)

,checking_account,"coalesce(t.checking_account,'no_info')"
0,None,no_info
1,little,little
2,None,no_info
3,None,no_info
4,None,no_info
...,...,...
995,little,little
996,moderate,moderate
997,moderate,moderate
998,None,no_info


In [18]:
t = pd.DataFrame({'id':[1,1,2],'name':['a','a','b']})

In [19]:
t.to_sql('dupl_test',con,index=False,if_exists='replace')

3

In [20]:
sql = '''select * from dupl_test t'''
select(sql)

,id,name
0,1,a
1,1,a
2,2,b


In [22]:
sql = '''select t.id, t.name, count(1) as cnt from dupl_test t
group by t.id, t.name
having count(1) > 1
'''
select(sql)

,id,name,cnt
0,1,a,2


In [23]:
t = pd.DataFrame({'id':[1,1,2,2,3],
                  'name':['a','b','c','d','e']})

In [24]:
t.to_sql('dupl_test',con,index=False,if_exists='replace')

5

In [25]:
sql = '''select t.id as cnt from dupl_test t
group by t.id
having count(1) > 1
'''
select(sql)

,cnt
0,1
1,2


In [26]:
sql = '''select * from dupl_test t
where t.id in (select t.id as cnt from dupl_test t
group by t.id
having count(1) > 1)'''
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d


In [27]:
sql = '''select date(t.contract_dt,'start of month') as month,
count(1) as credit_cnt,
count(distinct t.client_id) as client_id_unique,
sum(t.credit_amount) as credit_amount_sum,
avg(t.credit_amount) as credit_amount_avg

 from german_credit t
 group by date(t.contract_dt,'start of month')
 order by date(t.contract_dt,'start of month')
 '''
select(sql)

,month,credit_cnt,client_id_unique,credit_amount_sum,credit_amount_avg
0,2007-05-01,81,81,207663,2563.740741
1,2007-06-01,74,74,239594,3237.756757
2,2007-07-01,71,71,224333,3159.619718
3,2007-08-01,57,57,178569,3132.789474
4,2007-09-01,58,58,186909,3222.568966
5,2007-10-01,70,70,188534,2693.342857
6,2007-11-01,87,87,300504,3454.068966
7,2007-12-01,77,77,273973,3558.090909
8,2008-01-01,93,93,288080,3097.634409
9,2008-02-01,55,55,211128,3838.690909


In [28]:
sql = '''select t.credit_amount,

case when t.credit_amount < 1000 then '1. <1000'
when t.credit_amount < 2000 then '2. 1000-2000'
when t.credit_amount < 3000 then '3. 2000-3000'
when t.credit_amount >= 3000 then '4. >= 3000'

else 'other' end as credit_amount_bin

 from german_credit t'''

select(sql)

,credit_amount,credit_amount_bin
0,3074,4. >= 3000
1,1344,2. 1000-2000
2,936,1. <1000
3,1393,2. 1000-2000
4,776,1. <1000
...,...,...
995,2600,3. 2000-3000
996,4455,4. >= 3000
997,6403,4. >= 3000
998,5003,4. >= 3000


In [29]:
sql = '''select

case
when t.credit_amount < 1000 then '1. <1000'
when t.credit_amount < 2000 then '2. 1000-2000'
when t.credit_amount < 3000 then '3. 2000-3000'
when t.credit_amount >= 3000 then '4. >= 3000'

else 'other' end as credit_amount_bin,

count(1) as credit_cnt

 from german_credit t
 group by 1
 '''

select(sql)

,credit_amount_bin,credit_cnt
0,1. <1000,116
1,2. 1000-2000,316
2,3. 2000-3000,188
3,4. >= 3000,380


In [30]:
sql = '''select t.housing,

count(case when t.sex = 'female' then 1 else null end) as female,
count(case when t.sex = 'male' then 1 else null end) as male,

count(1) as cnt

from german_credit t
group  by t.housing'''

select(sql)

,housing,female,male,cnt
0,free,19,89,108
1,own,196,517,713
2,rent,95,84,179


In [31]:
sql = '''select distinct t.purpose from german_credit t'''
purpose = list(select(sql)['purpose'].values)
purpose

['radio/TV',
 'car',
 'education',
 'furniture/equipment',
 'repairs',
 'business',
 'domestic appliances',
 'vacation/others']

In [32]:
for p in purpose:
  print(f"count(case when t.purpose = '{p}' then 1 else null end) as {p.lower().replace(' ','').replace('/','')},")

count(case when t.purpose = 'radio/TV' then 1 else null end) as radiotv,
count(case when t.purpose = 'car' then 1 else null end) as car,
count(case when t.purpose = 'education' then 1 else null end) as education,
count(case when t.purpose = 'furniture/equipment' then 1 else null end) as furnitureequipment,
count(case when t.purpose = 'repairs' then 1 else null end) as repairs,
count(case when t.purpose = 'business' then 1 else null end) as business,
count(case when t.purpose = 'domestic appliances' then 1 else null end) as domesticappliances,
count(case when t.purpose = 'vacation/others' then 1 else null end) as vacationothers,


In [33]:
sql = '''select t.housing,

count(case when t.purpose = 'radio/TV' then 1 else null end) as radiotv,
count(case when t.purpose = 'car' then 1 else null end) as car,
count(case when t.purpose = 'education' then 1 else null end) as education,
count(case when t.purpose = 'furniture/equipment' then 1 else null end) as furnitureequipment,
count(case when t.purpose = 'repairs' then 1 else null end) as repairs,
count(case when t.purpose = 'business' then 1 else null end) as business,
count(case when t.purpose = 'domestic appliances' then 1 else null end) as domesticappliances,
count(case when t.purpose = 'vacation/others' then 1 else null end) as vacationothers,
count(1) as cnt

from german_credit t
group  by t.housing'''
select(sql)

,housing,radiotv,car,education,furnitureequipment,repairs,business,domesticappliances,vacationothers,cnt
0,free,15,55,15,11,3,5,0,4,108
1,own,227,219,34,122,17,76,10,8,713
2,rent,38,63,10,48,2,16,2,0,179
